# Load data

In [ ]:
#%matplotlib qt
import numpy as np
import pandas as pd
import os
# import keyboard
import matplotlib.pyplot as plt

from utils import * #ecef_to_llh, llh_to_ecef
from read_user import User
from read_obs import read_real_rover_obs, read_obs_data
from read_base import read_real_base_obs, parse_rinex_observation_data
# from EKF_utils import *


In [ ]:
dir_path = os.getcwd()
data =  'RWD_20250422/D1T4' #'RWD_20250422/D1T4' 'YJN\Tightly_coupled_EKF-main' 'RWD_20250422/D2T10'
folder_path = os.path.join(dir_path, data)

''' Get User Position, IMU'''#读取 用户位置和IMU数据
user = User()
user.read_real_user_data(os.path.join(folder_path, 'user_pos.csv'), os.path.join(folder_path, 'imu_data.csv'))
origin = user.pos_llh[0, :] 

''' Read Base Observations '''#读取基站观测数据
base_obs = read_real_base_obs(os.path.join(folder_path, 'base_obs.csv'))

if 'D1' in data: # h + 45.5
    base_position, base_ecef = [48.0779195, 11.6305769, 549.51], False
    lte_tx1_llh = [48.0780113, 11.6307491, 549.40]
    lte_tx2_llh = [48.0778041, 11.6306888, 549.38]
    pl_tx1_llh = [48.0779030, 11.6309295, 551.38]
    pl_tx2_llh = [48.0781220, 11.6306068, 551.25]

elif 'D2' in data:
    base_position, base_ecef = [48.0779172, 11.6305752, 549.57], False
    lte_tx1_llh = [48.0780048, 11.6307446, 549.40]
    lte_tx2_llh = [48.0777902, 11.6306752, 549.42]
    pl_tx1_llh  = [48.0778883, 11.6309177, 551.56]
    pl_tx2_llh  = [48.0781255, 11.6306180, 551.40]
else:
    print("\033[31mError\033[0m: No transmitter data available for this dataset.")

lte_tx1_ecef = llh_to_ecef([lte_tx1_llh])[0]
lte_tx2_ecef = llh_to_ecef([lte_tx2_llh])[0]
pl_tx1_ecef = llh_to_ecef([pl_tx1_llh])[0]
pl_tx2_ecef = llh_to_ecef([pl_tx2_llh])[0]
TX_POS = {'L01': lte_tx1_ecef, 'L02': lte_tx2_ecef, 'P47': pl_tx1_ecef, 'P48': pl_tx2_ecef}

''' Read Rover Observations '''
rover_gnss = read_real_rover_obs(os.path.join(folder_path, 'rover_obs.csv'), TX_POS)

# print(pd.read_csv(os.path.join(folder_path, 'rover_obs_crop.csv'))['id'].unique())
# print(pd.read_csv(os.path.join(folder_path, 'rover_obs_crop.csv'))['sv_id'].unique())

# Get same time
np.set_printoptions(precision=10, suppress=False)
user_times = np.round(np.asarray(user.user['gps_time'].values, dtype=float), 1)
rover_obs_time = np.round(np.array(list(rover_gnss.keys()), dtype=float), 1)
overlapped_time, idx1, idx2 = np.intersect1d(user_times, rover_obs_time, return_indices=True)
# print(user_times[:10])
# print(rover_obs_time[:10])
# print(overlapped_time)
print(len(idx1), len(idx2))
print(max(user_times) - min(user_times), max(rover_obs_time) - min(rover_obs_time))
print("User time range:", min(user_times), max(user_times))
print("Rover time range:", min(rover_obs_time), max(rover_obs_time))
print("Overlapped time range:", min(overlapped_time), max(overlapped_time))

716 716
111.0 83.10000014305115
User time range: 1429364521.4 1429364632.4
Rover time range: 1429364549.3 1429364632.4
Overlapped time range: 1429364549.3 1429364632.4


In [3]:
unique_ids = {obs.id for obs_list in rover_gnss.values() for obs in obs_list}
print(sorted(unique_ids))

# print(user.user['gps_time'].values[:50]/1e9)


['E05', 'E09', 'E15', 'E18', 'E21', 'E27', 'E30', 'E34', 'E36', 'G01', 'G02', 'G08', 'G10', 'G15', 'G16', 'G23', 'G27', 'G32', 'L01', 'L02', 'P47', 'P48']


In [4]:
# df = pd.read_csv(os.path.join(dir_path, 'learning/real_with_predicted_multipath.csv'))
df = pd.read_csv(os.path.join(dir_path, 'learning/real_with_predicted_multipath_svm_bal.csv'))
df = df.dropna()

print(df.head())



       gps_time satellite_id  sv_id   pseudorange  doppler_shift        cn0  \
0  1.429365e+09          G01      1  2.371945e+07   -3039.008006  42.975460   
1  1.429365e+09          G02      2  2.160127e+07   -1837.173477  45.609964   
2  1.429365e+09          G08      8  2.007511e+07    -926.014036  48.385704   
3  1.429365e+09          G10     10  2.069873e+07     849.496573  48.685554   
4  1.429365e+09          G23     23  2.277589e+07    3131.674603  39.140482   

      azimuth  elevation  pseudorange_residual  pseudorange_corrected_cb  \
0  262.633921  15.781663             -0.469555              2.409778e+07   
1  277.053016  44.457877              0.041923              2.185358e+07   
2  290.422082  66.985774             -0.611992              2.053719e+07   
3   76.044600  65.665333              0.836119              2.088786e+07   
4   47.377842  25.368725             -4.558627              2.322375e+07   

         sat_px        sat_py        sat_pz  sat_type    cn0_new  \


In [5]:
# los_mask = df['predicted_multipath'] == 0
# nlos_mask = df['predicted_multipath'] == 1
gps_pl_mask = (df['sat_type'] == 0) | (df['sat_type'] == 1)
los_mask = (
    (df['predicted_multipath'] == 0) &
    ((df['sat_type'] == 0) | (df['sat_type'] == 1))
)
nlos_mask = (
    (df['predicted_multipath'] == 1) &
    ((df['sat_type'] == 0) | (df['sat_type'] == 1))
)

print("Total observations:", len(df))
print("GPS+PL observations:", len(df[gps_pl_mask]), " / ", len(df))
print("LOS observations:", len(df[los_mask]), " / ", len(df))
print("NLOS observations:", len(df[nlos_mask]), " / ", len(df))

# print(df.loc[los_mask, 'gps_time'].values[:50])

nlos_dict = (
    df.loc[df['predicted_multipath'] == 1]
      .groupby('gps_time')['satellite_id']
      .apply(list)       # or set
      .to_dict()
)

print(nlos_dict)

from collections import Counter

sat_count = Counter(sat_id for sats in nlos_dict.values() for sat_id in sats)
print(sat_count)

# rover_time = np.round(np.fromiter(rover_gnss.keys(), dtype=float), 1)
# for time, sats in nlos_dict.items():
#     if time not in rover_time:
#         print(f"Time {time} not in rover_gnss")
#         continue
#     idx = np.where(rover_time == time)[0]
#     key_list = list(rover_gnss.keys())
#     obs_key = key_list[idx[0]]   # since idx is usually a 1-element array
#     obs_list = rover_gnss[obs_key]
#     for sat in sats:
#         if sat not in [obs.id for obs in obs_list]:
#             print(f"Satellite {sat} not in observations at time {time}")
#             continue

# rover_gnss_los = {}

# for t, obs_list in rover_gnss.items():
#     t = round(t, 1)  # ensure matching precision
#     if t in nlos_dict:
#         nlos_sats = set(nlos_dict[t])
#         filtered_obs = [obs for obs in obs_list if obs.id not in nlos_sats]
#     else:
#         filtered_obs = obs_list  # no NLOS → keep all
#     rover_gnss_los[t] = filtered_obs

# print(df['gps_time'].unique())
# print(unique_times = df['gps_time'].unique())



Total observations: 11243
GPS+PL observations: 6376  /  11243
LOS observations: 4329  /  11243
NLOS observations: 2047  /  11243
{1429364549.3: ['G23'], 1429364549.4: ['G23'], 1429364549.5: ['G23'], 1429364549.6: ['G23'], 1429364549.7: ['G23'], 1429364549.8: ['G23'], 1429364549.9: ['G23'], 1429364550.2: ['G23'], 1429364550.3: ['G23'], 1429364550.4: ['G23'], 1429364550.5: ['G23'], 1429364550.6: ['G23'], 1429364550.7: ['G23'], 1429364550.8: ['G23'], 1429364550.9: ['G23'], 1429364551.0: ['G23'], 1429364551.2: ['G23'], 1429364551.3: ['G23'], 1429364551.4: ['G23'], 1429364551.5: ['G23', 'G32', 'P48'], 1429364551.6: ['G23', 'G32', 'P48'], 1429364551.7: ['G23', 'G32', 'P48'], 1429364551.8: ['G23', 'G32', 'P48'], 1429364551.9: ['G23', 'G32', 'P48'], 1429364552.0: ['G23', 'G32', 'P48'], 1429364552.2: ['G23', 'G32', 'P48'], 1429364552.3: ['G23', 'G32', 'P48'], 1429364552.4: ['G23', 'G32', 'P48'], 1429364552.5: ['G23', 'G32', 'P48'], 1429364552.6: ['G23', 'G32', 'P48'], 1429364552.7: ['G23', 'G32

In [6]:
CASES = {
    1 : [['C', 'E', 'P', 'L'], False],
    2 : [['C', 'E', 'L'], False],
    3 : [['C', 'E', 'P', 'L'], True],
    4 : [['C', 'E', 'L'], True],
}

sat_per_time = (
    df.groupby("gps_time")["satellite_id"]
      .apply(list)   # or .apply(set) if you want unique only
      .to_dict()
)

nlos_dict = (
    df.loc[df['predicted_multipath'] == 1]
      .groupby('gps_time')['satellite_id']
      .apply(list)       # or set
      .to_dict()
)



results = {}
for i, (exclude_id, exclude_NLOS) in CASES.items():
    estimated_ecef, estimated_cb, data = {}, {}, {}
    for time, sats in sat_per_time.items():
        data_obs, sat_positions, pseudoranges = [], [], []
        for sat in sats:
            if any(ex_id in sat for ex_id in exclude_id):
                continue
            if exclude_NLOS and time in nlos_dict and sat in nlos_dict[time]:
                print(f"Case {i}: Time {time}: Excluding NLOS satellite {sat}.")
                continue
            row = df[(df["gps_time"] == time) & (df["satellite_id"] == sat)].iloc[0]
            # print(row["sat_pos"], type(row["sat_pos"]))
            # split = first_satpos.replace('[','').replace(']','').split(',')
            # sat_positions.append([float(k) for k in row["sat_pos"].replace('[','').replace(']','').split(',')])
            sat_positions.append([float(row['sat_px']), float(row['sat_py']), float(row['sat_pz'])])
            pseudoranges.append(float(row["pseudorange_corrected_cb"]))
        if len(sat_positions) < 4:
            print(f"Case {i}: Time {time}: Not enough satellites after filtering. Needed 4, got {len(sat_positions)}.")
            continue
        # sat_positions = np.array(sat_positions, dtype=float).reshape(-1, 3)
        # pseudoranges = np.array(pseudoranges, dtype=float)
        receiver_position, clock_bias = single_point_positioning(
                sat_positions, pseudoranges, np.append(user.pos_ecef[0], 0)) 
        estimated_ecef[time] = receiver_position
        estimated_cb[time] = clock_bias

    overlapped_time, idx1, idx2 = np.intersect1d(user_times, np.array(list(estimated_ecef.keys())), return_indices=True)

    estimated_enu = {
            k: llh_to_enu(ecef_to_llh([v]), origin)
            for k, v in estimated_ecef.items() }
    
    results[i] = {
        'estimated_ecef': estimated_ecef, 
        'estimated_cb': estimated_cb,
        'estimated_enu': estimated_enu,
        'overlapped_time': overlapped_time,
        'idx1': idx1, # User time indices
        'idx2': idx2 # GNSS time indices
    }



            

    




Case 1: Time 1429364549.3: Not enough satellites after filtering. Needed 4, got 2.
Case 1: Time 1429364549.4: Not enough satellites after filtering. Needed 4, got 2.
Case 1: Time 1429364549.5: Not enough satellites after filtering. Needed 4, got 2.
Case 1: Time 1429364549.6: Not enough satellites after filtering. Needed 4, got 2.
Case 1: Time 1429364549.7: Not enough satellites after filtering. Needed 4, got 2.
Case 1: Time 1429364549.8: Not enough satellites after filtering. Needed 4, got 2.
Case 1: Time 1429364549.9: Not enough satellites after filtering. Needed 4, got 2.
Case 1: Time 1429364550.2: Not enough satellites after filtering. Needed 4, got 2.
Case 1: Time 1429364550.3: Not enough satellites after filtering. Needed 4, got 2.
Case 1: Time 1429364550.4: Not enough satellites after filtering. Needed 4, got 2.
Case 1: Time 1429364550.5: Not enough satellites after filtering. Needed 4, got 2.
Case 1: Time 1429364550.6: Not enough satellites after filtering. Needed 4, got 2.
Case

In [ ]:
user_enu = llh_to_enu(user.pos_llh, origin)

for i in results.keys():
    print(f"Results for case {i}:")
    print(f"Estimated ECEF positions: {len(results[i]['estimated_ecef'])}")
    keys = list(results[i]['estimated_enu'].keys())
    enu_array = np.array([np.squeeze(results[i]['estimated_enu'][k]) for k in keys])
    error = np.linalg.norm(enu_array[results[i]['idx2'], :2] - user_enu[results[i]['idx1'], :2], axis=1)
    print("2D Error in enu:", np.mean(error))
    error = np.linalg.norm(enu_array[results[i]['idx2'], :] - user_enu[results[i]['idx1'], :], axis=1)
    print("3D Error in enu:", np.mean(error))
    print("\n")

#%matplotlib qt
plot_trajectory(
    {'user': user_enu,
        **{i: np.array([np.squeeze(v) for v in results[i]['estimated_enu'].values()])
            for i in results.keys()} }, 'Trajectories', 'enu', twoD=True, scatter=True
)

plt.show()

Results for case 1:
Estimated ECEF positions: 697
2D Error in enu: 3.2631127376153586
3D Error in enu: 5.066412280713907


Results for case 2:
Estimated ECEF positions: 697
2D Error in enu: 3.3074822153855723
3D Error in enu: 3.9742629424159235


Results for case 3:
Estimated ECEF positions: 543
2D Error in enu: 8.573265043504955
3D Error in enu: 11.865798730586336


Results for case 4:
Estimated ECEF positions: 690
2D Error in enu: 5.1031871234586985
3D Error in enu: 6.390787139005938


